
# Semantic Prompt Matching Application

This notebook guides you through the creation of a simple program that matches a new user prompt to the semantically closest pre-defined prompt and returns the associated response. This is useful for building basic chatbots or automated Q&A systems.

## Steps Involved:

1. **Data Preparation**: Define a list of pre-defined prompts and their associated responses.
2. **Semantic Similarity Calculation**: Implement a method to calculate the semantic similarity between the new prompt and each pre-defined prompt.
3. **Finding the Closest Match**: Determine which pre-defined prompt is semantically closest to the new prompt.
4. **Returning the Response**: Return the response associated with the closest pre-defined prompt.

Let's get started!



## 1. Data Preparation

First, we need to prepare our dataset. This includes defining a set of pre-defined prompts and their corresponding responses. We'll store these in a Python dictionary for easy access.


In [ ]:

# Sample data: pre-defined prompts and responses
predefined_prompts = {
    "What is your name?": "I'm a chatbot created to help you.",
    "How are you?": "I'm just a program, so I don't have feelings, but thanks for asking!",
    "Tell me a joke.": "Why don't scientists trust atoms? Because they make up everything!"
}



## 2. Semantic Similarity Calculation (Updated)

Instead of using TF-IDF, we will now use a more advanced method involving machine learning embeddings for better semantic understanding. We'll use the Universal Sentence Encoder from TensorFlow Hub to convert sentences into embeddings and then calculate cosine similarity.

First, ensure you have TensorFlow and TensorFlow Hub installed:
```python
!pip install tensorflow tensorflow-hub
```

Then, load the Universal Sentence Encoder model:
```python
import tensorflow_hub as hub

# Load the model
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
```

Now, we'll update our function to calculate semantic similarity using these embeddings:


In [ ]:

import tensorflow_hub as hub
import numpy as np

# Load the Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Function to calculate semantic similarity using embeddings
def calculate_similarity(new_prompt, predefined_prompts):
    # Combine new prompt with predefined prompts
    prompts = list(predefined_prompts.keys()) + [new_prompt]
    
    # Generate embeddings for each prompt
    embeddings = embed(prompts)
    
    # Calculate cosine similarity
    similarities = np.inner(embeddings[-1], embeddings[:-1])
    return similarities


In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Function to calculate semantic similarity
def calculate_similarity(new_prompt, predefined_prompts):
    # Combine new prompt with predefined prompts
    prompts = list(predefined_prompts.keys()) + [new_prompt]
    
    # Convert prompts to TF-IDF representation
    vectorizer = TfidfVectorizer().fit(prompts)
    prompt_vectors = vectorizer.transform(prompts)

    # Calculate cosine similarity
    similarities = cosine_similarity(prompt_vectors[-1], prompt_vectors[:-1]).flatten()
    return similarities



## 3. Finding the Closest Match

After calculating the similarities, we need to find the pre-defined prompt that is most similar to the new prompt.


In [ ]:

# Function to find the closest match
def find_closest_match(new_prompt, predefined_prompts):
    similarities = calculate_similarity(new_prompt, predefined_prompts)
    closest_index = similarities.argmax()
    closest_prompt = list(predefined_prompts.keys())[closest_index]
    return closest_prompt



## 4. Returning the Response

Finally, we return the response associated with the closest pre-defined prompt.


In [ ]:

# Function to get response for a new prompt
def get_response(new_prompt, predefined_prompts):
    closest_prompt = find_closest_match(new_prompt, predefined_prompts)
    return predefined_prompts[closest_prompt]

# Example
new_prompt = "Tell me something funny."
response = get_response(new_prompt, predefined_prompts)
print("Response:", response)



## Interactive Prompt and Response

To make our application interactive, we'll add an input field for entering a new prompt and a function to display the associated response.


In [ ]:

import ipywidgets as widgets
from IPython.display import display, clear_output

# Text input widget for the new prompt
prompt_input = widgets.Text(
    value='',
    placeholder='Type your prompt here',
    description='Prompt:',
    disabled=False
)

# Button to submit the prompt
submit_button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='', 
    tooltip='Click to get response',
    icon='check' 
)

# Output widget to display the response and similarity score
output = widgets.Output()

# Minimum similarity score threshold
min_similarity_threshold = 0.5  # Adjust this threshold as needed

# Function to handle button click event
def on_submit_clicked(b):
    with output:
        clear_output(wait=True)
        if prompt_input.value:
            similarities = calculate_similarity(prompt_input.value, predefined_prompts)
            max_similarity = max(similarities)
            if max_similarity < min_similarity_threshold:
                print("Sorry, I don't know how to respond to that.")
            else:
                closest_prompt = find_closest_match(prompt_input.value, predefined_prompts)
                response = predefined_prompts[closest_prompt]
                similarity_score = round(max_similarity * 100, 2)  # Convert to percentage
                print(f"Response: {response}"
Similarity Score: {similarity_score}")

# Event handler for the button
submit_button.on_click(on_submit_clicked)

# Display the widgets
display(prompt_input, submit_button, output)
